# Kaggle quick test (Qwen 8B + BGE-m3)
- Cài dependency, chuyển val/test CSV sang JSON, gọi `predict.py` để sinh `submission.csv`.
- Mặc định dùng index `kb_bge_m3_index.pkl` đi kèm repo và model `Qwen/Qwen3-8B-Base`.

In [ ]:
!pip install -q -r requirements.txt pandas

In [ ]:
# ==== Config paths (chỉnh lại theo Dataset bạn mount vào Kaggle) ====
VAL_PATH = "/kaggle/input/your-dataset/val.csv"   # để trống nếu không có
TEST_PATH = "/kaggle/input/your-dataset/test.csv"
BGE_INDEX_PATH = "kb_bge_m3_index.pkl"
MODEL_ID = "Qwen/Qwen3-8B-Base"
TOP_K = 6

In [ ]:
import json, os, tempfile
import pandas as pd

# Heuristic converter CSV -> JSON format predict.py cần

def _pick_column(df, targets):
    for t in targets:
        for c in df.columns:
            if c.lower() == t:
                return c
    return None


def convert_csv_to_json(csv_path: str, out_path: str):
    df = pd.read_csv(csv_path)
    qid_col = _pick_column(df, ["qid", "id"])
    q_col = _pick_column(df, ["question", "prompt"])
    if q_col is None:
        raise ValueError("Không tìm thấy cột question trong CSV")
    choice_cols = [c for c in df.columns if c.lower() in ["a", "b", "c", "d", "e", "f"]]
    if not choice_cols:
        # fallback option_* pattern
        choice_cols = [c for c in df.columns if c.lower().startswith("option")]
    if not choice_cols:
        raise ValueError("Không tìm thấy cột lựa chọn trong CSV")
    records = []
    answers = {}
    ans_col = _pick_column(df, ["answer", "label"])
    for idx, row in df.iterrows():
        qid = str(row[qid_col]) if qid_col else str(idx)
        qtext = str(row[q_col])
        choices = []
        for c in choice_cols:
            val = row.get(c)
            if pd.isna(val):
                continue
            choices.append(str(val))
        records.append({"qid": qid, "question": qtext, "choices": choices})
        if ans_col:
            ans = str(row[ans_col]).strip().upper()
            if ans:
                answers[qid] = ans
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)
    return out_path, answers if answers else None


def prepare_input(path: str, tag: str):
    if not path:
        return None, None
    if path.lower().endswith(".json"):
        return path, None
    out_path = os.path.join("/kaggle/working", f"{tag}.json")
    return convert_csv_to_json(path, out_path)

In [ ]:
# ==== Chuẩn bị dữ liệu ====
val_input, val_answers = prepare_input(VAL_PATH, "val_prepared")
test_input, _ = prepare_input(TEST_PATH, "test_prepared")
print("val_input:", val_input)
print("test_input:", test_input)

In [ ]:
# ==== Chạy inference trên val (nếu có) ====
if val_input:
    !ENABLE_THINKING=1 LOCAL_LLM_MODEL_ID={MODEL_ID}       python predict.py --input {val_input} --output val_pred.csv --bge_index {BGE_INDEX_PATH} --top_k_retrieval {TOP_K}
else:
    print("Bỏ qua val (không cấu hình VAL_PATH)")

In [ ]:
# ==== Đánh giá nhanh val nếu có ground-truth ====
if val_input and val_answers:
    pred = pd.read_csv("val_pred.csv")
    pred_dict = dict(zip(pred.qid.astype(str), pred.answer.astype(str)))
    total = len(val_answers)
    correct = sum(1 for k, v in val_answers.items() if pred_dict.get(k, "") == v)
    print(f"Accuracy: {correct}/{total} = {correct/total:.4f}")
else:
    print("Không có nhãn val để tính accuracy")

In [ ]:
# ==== Chạy inference trên test và xuất submission.csv ====
if test_input:
    !ENABLE_THINKING=1 LOCAL_LLM_MODEL_ID={MODEL_ID}       python predict.py --input {test_input} --output submission.csv --bge_index {BGE_INDEX_PATH} --top_k_retrieval {TOP_K}
    import pandas as pd
    print(pd.read_csv("submission.csv").head())
else:
    print("Chưa cấu hình TEST_PATH")